<a href="https://colab.research.google.com/github/afran90/Soft_computing_Codes1/blob/master/implement_of_viterbi_algo_using_hmm_with_fair_bias_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LOAD INPUT
HERE SAMPLE_DATASET TAKES TRAINING DATA AS AN INPUT

In [0]:
def input_data(sample_dataset):
#pandas conversion 
  print('Upload Training Data',end='\n')
  import numpy as np
  import pandas as pd

  from google.colab import files
  uploaded=files.upload()

  import io
  dataset=pd.read_csv(io.BytesIO(uploaded[sample_dataset]),sep="\t",header=None)


  dataset.columns=['Game_Condition','Result']
  
  #print(dataset)

  return dataset



FOR EMISSION MATRIX
>CALCULATE THE PROBABILITY OF DIFFERENT TEST SAMPLE AND STORED IN EMMISION MATRIX

In [0]:
def emission_matrix_calculate(trainData):
  print('Now in Emission_Matrix Function:',end='\n')

  import numpy as np
  import pandas as pd

  #Calculate Tota;_cheat and Total_Fair
  s=trainData.groupby('Game_Condition').count()
  print(s)
  #print(type(s))
  total_cheat=s['Result'][0]
  total_fair=s['Result'][1]

  print('Total Cheat=',total_cheat,'Total_Fair=',total_fair)

  s=trainData.groupby("Game_Condition")["Result"].value_counts()
  print(s)
  print(s[0],s[1],s[2],s[3])
#Emission Matrix
  Emission_Matrix=[[],[]]
  Emission_Matrix[0].append(s[0]/total_cheat)
  Emission_Matrix[0].append(s[1]/total_cheat)

  Emission_Matrix[1].append(s[2]/total_fair)
  Emission_Matrix[1].append(s[3]/total_fair)

  
  Emission_Matrix=np.array(Emission_Matrix)
  Emission_Matrix

#Log

  Emission_Matrix=np.log2(Emission_Matrix)
  print('Emission Matrix',end='\n')
  print(Emission_Matrix)


  return Emission_Matrix


FOR TRANSITION MATRIX
> CALCULATE THE PROBABILITY OF DIFFERENT STATE TRANSITION FOR TRAINING DATA
 AND VALUE STORED IN TRANSITION MATRIX FOR FURTHER CALCULATION


In [0]:
def transition_matrix_count(trainData):
  print('Now in Transition_Matrix Function:',end='\n')
  import numpy as np
  import pandas as pd

   #Calculate Total_cheat and Total_Fair
  s=trainData.groupby('Game_Condition').count()
  print(s)
  #print(type(s))
  total_cheat=s['Result'][0]
  total_fair=s['Result'][1]

  print('Total Cheat=',total_cheat,'Total_Fair=',total_fair)

      #Transition Matrix
  dummy=trainData
  #drop column 
  dummy=dummy.drop(['Result'],axis=1)

  dummy['Next_Game_Condition']=dummy['Game_Condition'].shift(-1)
  Modified_dataset=dummy
  print(Modified_dataset)

  #Transition Matrix
  s=Modified_dataset.groupby("Game_Condition")["Next_Game_Condition"].value_counts()
  print(s)
  print(s[0],s[1],s[2],s[3])

  Transition_Matrix=[[],[]]

  Transition_Matrix[0].append(s[0]/(total_cheat))#As for last data sample we dont know the next condition..and the last data sample is cheat.So 1 is subtracted
  Transition_Matrix[0].append(s[1]/(total_cheat))
  Transition_Matrix[1].append(s[2]/(total_fair-1))
  Transition_Matrix[1].append(s[3]/(total_fair-1))

  
  Transition_Matrix=np.array(Transition_Matrix)
  print('Transition Matrix',end='\n')
#log
  Transition_Matrix=np.log2(Transition_Matrix)
  print(Transition_Matrix)
  return Transition_Matrix

INPUT_OBSERVATION FUNCTION TAKES TESTING DATA SET

In [0]:
def input_observation(testingData):
  print('Upload Testing Data',end='\n')
  from google.colab import files
  uploaded=files.upload()
  import io
  import numpy as np
  import pandas as pd
  

  testing_dataset=pd.read_csv(io.BytesIO(uploaded[testingData]),sep="\t",header=None)

  return testing_dataset

IMPLEMENT HMM USING THE VITERBI ALGORITHM
>Reference:
Murphy, Kevin P. Machine Learning: A Probabilistic Perspective. MIT press, 2012.

![alt text](https://drive.google.com/uc?id=1J9yuHGxAKhbX67x_lmFylPzeZAKkm7Ns)

In [0]:
def viterbi(testing_dataset, Emission_Matrix, Intial_Matrix, Transition_Matrix):
  
  print('Now in Viterbi Function:',end='\n')

  import io
  import numpy as np
  testing_sample_size=testing_dataset.size
  print(testing_sample_size)

  Fair=np.zeros(testing_sample_size)
  Cheat=np.zeros(testing_sample_size)

  #Intiial Values of Fair And Cheat

#Emission_Matrix[0][0]=Condition=Cheat,Result=Lose
#Emission_Matrix[0][1]=Condition=Cheat,Result=Win
#Emission_Matrix[1][0]=Condition=Fair,Result=Win
#Emission_Matrix[1][1]=Condition=Fair,Result=Lose


  Initial_Sample=testing_dataset[0][0]

  if (Initial_Sample=='win'):
    Fair[0]=np.add(Intial_Matrix[1],Emission_Matrix[1][0])
    Cheat[0]=np.add(Intial_Matrix[0],Emission_Matrix[0][1])


  if(Initial_Sample=='lose'):
    Fair[0]=np.add(Intial_Matrix[1],Emission_Matrix[1][1])
    Cheat[0]=np.add(Intial_Matrix[1],Emission_Matrix[0][0])


  for i in range(1,testing_sample_size):
    dummy_fair_fair=1
    dummy_fair_cheat=1
    dummy_cheat_cheat=1
    dummy_cheat_fair=1

    Next_State=testing_dataset[0][i]
    
#Transition_Matrix[0][0]=Present Condition=Cheat,Next Condition=Cheat

#Transition_Matrix[0][1]=Present Condition=Cheat,Next Condition=Fair

#Transition_Matrix[1][0]=Present Condition=Fair,Next Condition=Fair

#Transition_Matrix[1][1]=Present Condition=Fair,Next Condition=Cheat

  #considering next condition is Fair

    if(Next_State=='win'):
    
      dummy_fair_fair=np.add(np.add(Fair[i-1],Transition_Matrix[1][0]),Emission_Matrix[1][0])
      dummy_cheat_fair=np.add(np.add(Cheat[i-1],Transition_Matrix[0][1]),Emission_Matrix[1][0])
  
      Fair[i]=np.maximum(dummy_fair_fair,dummy_cheat_fair)    
  
    if(Next_State=='lose'):
      dummy_fair_fair=np.add(np.add(Fair[i-1],Transition_Matrix[1][0]),Emission_Matrix[1][1])
      dummy_cheat_fair=np.add(np.add(Cheat[i-1],Transition_Matrix[0][1]),Emission_Matrix[1][1])

      Fair[i]=np.maximum(dummy_fair_fair,dummy_cheat_fair)
  
  #considering next condition is Cheat

    if(Next_State=='win'):
      dummy_cheat_cheat=np.add(np.add(Cheat[i-1],Transition_Matrix[0][0]),Emission_Matrix[0][1])
      dummy_fair_cheat=np.add(np.add(Fair[i-1],Transition_Matrix[1][1]),Emission_Matrix[0][1])

      Cheat[i]=np.maximum(dummy_cheat_cheat,dummy_fair_cheat)

    if(Next_State=='lose'):
      dummy_cheat_cheat=np.add(np.add(Cheat[i-1],Transition_Matrix[0][0]),Emission_Matrix[0][0])
      dummy_fair_cheat=np.add(np.add(Fair[i-1],Transition_Matrix[1][1]),Emission_Matrix[0][0])

      Cheat[i]=np.maximum(dummy_cheat_cheat,dummy_fair_cheat)

  print(Cheat.size)
  print(Fair.size)


  Output_Matrix=[]
  for i in range(testing_sample_size):
    if(Fair[i]>Cheat[i]):
      Output_Matrix.append('Fair')
    if(Fair[i]<Cheat[i]):
      Output_Matrix.append('Cheat')  

  Output_Matrix=np.array(Output_Matrix)
  print(Output_Matrix.size)
  #Resulting dataframe
  Resulting_dataset=testing_dataset
  Resulting_dataset.columns=['Result']
  Resulting_dataset['Condition']=Output_Matrix
  
  
  print(Resulting_dataset)

  s=Resulting_dataset.groupby("Result")["Condition"].value_counts()
  print(s)

  #print Fair and Cheat

  return Resulting_dataset  


MAIN FUNCTION

In [0]:
def main():

    import numpy as np
    import pandas as pd

    print('Now in Main Function:',end='\n')

    sample_dataset = "training_Pyramid_1000.data.txt"  
    testing_data = "testing_Pyramid_1000.data.txt"     
   
    trainData = input_data(sample_dataset)
    observations = input_observation(testing_data)
    initial_pro = np.array([0.5, 0.5])
    initial_pro=np.log2(initial_pro)


    Emission_Matrix = emission_matrix_calculate(trainData)
    Transition_Matrix = transition_matrix_count(trainData)

    print("Transition Matrix\n", Transition_Matrix)
    print("Emission Matrix\n", Emission_Matrix)
    print("Initial Matrix\n", initial_pro)



    #score, final_output = viterbi(observations, emission_matrix, initial_pro, transition_matrix)
    final_output = viterbi(observations, Emission_Matrix, initial_pro, Transition_Matrix)

    print(final_output)

    final_output.to_csv('pyramid_resulting_dataset.csv')
    from google.colab import files
    files.download('pyramid_resulting_dataset.csv')

main()

Now in Main Function:
Upload Training Data


Saving training_Pyramid_1000.data.txt to training_Pyramid_1000.data (3).txt
Upload Testing Data


Saving testing_Pyramid_1000.data.txt to testing_Pyramid_1000.data (3).txt
Now in Emission_Matrix Function:
                Result
Game_Condition        
cheat              488
fair               512
Total Cheat= 488 Total_Fair= 512
Game_Condition  Result
cheat           lose      343
                win       145
fair            win       274
                lose      238
Name: Result, dtype: int64
343 145 274 238
Emission Matrix
[[-0.50867257 -1.75082825]
 [-0.90196792 -1.10518224]]
Now in Transition_Matrix Function:
                Result
Game_Condition        
cheat              488
fair               512
Total Cheat= 488 Total_Fair= 512
    Game_Condition Next_Game_Condition
0             fair                fair
1             fair                fair
2             fair               cheat
3            cheat               cheat
4            cheat               cheat
..             ...                 ...
995           fair               cheat
996          cheat                fair
